#### 1.	 (5 points) Here in Champaign-Urbana, IL, the winter of 2015-2016 was noticeably warmer than the winter of 2014-2015. Below are some data on the maximum daily temperatures of ten randomly selected days from the winter of 2015-2016, and then the maximum daily temperatures from nine randomly selected days from 2014-2015. Below are these data.


In [88]:
source('../cspc440/R/utility.R')
temp_data <- read.csv(file = "../../temp_data.csv", header = T)

In [89]:
library(reshape2)
temp_data<-melt(temp_data, id.vars = c('Winter.Season', 'Winter.Season.1'),
                measure.vars = c('Maximum.Temp', 'Maximum.Temp.1'))
temp_data <- temp_data[,3:4]
colnames(temp_data)<-c("season", "max_temp")
temp_data$season <- factor(temp_data$season)
levels(temp_data$season) <- c("2015-2016", "2014-2015")

In [90]:
library(plyr)

ddply(.data = temp_data,.variables = .(season), summarise, 
      pval=shapiro.test(max_temp)$p.value)


,season,pval
1,2015-2016,0.9463874
2,2014-2015,0.4463203


We see that we fail to reject the null hypothesis of the shapiro wilk test and therefore we can use the two sample t-test

In [91]:
p <- t.test(x = temp_data[which(temp_data$season == "2015-2016"), "max_temp"],
       y = temp_data[which(temp_data$season == "2014-2015"), "max_temp"])$p.value
t <- t.test(x = temp_data[which(temp_data$season == "2015-2016"), "max_temp"],
       y = temp_data[which(temp_data$season == "2014-2015"), "max_temp"])$statistic

tab <- matrix(c(p,t), ncol=2,nrow=1,byrow=T)
colnames(tab) <- c("pval", "tstat")
tab

pval,tstat
0.00224618,4.00655551


Thus, we can reject the null hypothesis that the means are equal between the two populations.

In [92]:
p<-var.test(x = temp_data[which(temp_data$season == "2015-2016"), "max_temp"],
         y = temp_data[which(temp_data$season == "2014-2015"), "max_temp"])$p.value
F<-var.test(x = temp_data[which(temp_data$season == "2015-2016"), "max_temp"],
         y = temp_data[which(temp_data$season == "2014-2015"), "max_temp"])$statistic

tab <- matrix(c(p,F), ncol=2,nrow=1,byrow=T)
colnames(tab) <- c("pval", "Fstat")
tab

pval,Fstat
0.001465821,12.844387002


Thus we reject the null hypothesis that the variances are the same.

In [93]:
source('../cspc440/R/test_of_vars.R')
source('../cspc440/R/utility.R')

In [94]:
temp_2015<-temp_data[which(temp_data$season == "2015-2016"), "max_temp"]
temp_2014<-temp_data[which(temp_data$season == "2014-2015"), "max_temp"]

confint_var_ratio <- function(var1, var2, alpha){
  if(var(var1,na.rm = T)>var(var2,na.rm = T)){
    ratio <- var(var1, na.rm = T)/var(var2, na.rm=T)
    F_l <- qf(alpha/2, df1=length(var1)-1, df2=length(var2)-1)
    F_u <- qf(1-alpha/2,df1=length(var1)-1, df2=length(var2)-1)
  }else{
    ratio <- var(var2, na.rm=T)/var(var1,na.rm=T)
    F_l <- qf(alpha/2, df1=length(var2)-1, df2=length(var1)-1)
    F_u <- qf(1-alpha/2,df1=length(var2)-1, df2=length(var1)-1)

  }

  confint <- matrix(c(ratio*F_l ,  ratio*F_u), ncol=2, nrow=1, byrow=T)
  colnames(confint) <- c("lower $F_l\\times \\frac{s^2_1}{s^2_2}$", "upper $F_U\\times\\frac{s^2_1}{s^2_2}$")

  return(confint)
}

In [95]:
confint_var_ratio(temp_2014,temp_2015, alpha=0.05)

lower $F_l\times \frac{s^2_1}{s^2_2}$,upper $F_U\times\frac{s^2_1}{s^2_2}$
3.190364,51.711427


#### 1.	(5 points) A meteorologist studying the impact of el niño on winter temperatures is interested in assessing the temperatures during the winters of 2015-2016 in contrast to the previous two winters (i.e., the winters of 2014-2015 and 2013-2014). Therefore, this meteorologist has obtained maximum daily temperatures in Champaign-Urbana, IL from 15 randomly selected days from each of these three winter seasons. Below are these data.


In [96]:
temp_data_2 <- read.csv(file = '../../temp_data_2.csv', header = T)

#### a. (1 point) Suppose that the objective of this experiment is to determine whether or not the mean temperatures differed for these three winter seasons.  Write down the linear model for this experiment. Show all symbols, degrees of freedom and subscripts. Define each term in the model and state the assumptions made about the errors.


$$
Y_{ij} = \mu + \tau_i + \epsilon_{ij}
$$

$$
\mu, df=1, \text{  Population Grand Mean } 
$$

$$
\tau_i, df=t-1=2, \text{    Effect of the ith treatment   }
$$

$$
\epsilon_{ij} \overset{i.i.d.}{\sim} N(0,\sigma^2), df=n-t = 42, \text{    Random Residual Error }
$$



1. Use either SAS or R to determine if all 45 of these daily maximum temperatures collectively follow a normal distribution. Use α = 0.01.

In [97]:
p<-shapiro.test(x = temp_data_2$Maximum.Temp)$p.value
W<-shapiro.test(x = temp_data_2$Maximum.Temp)$statistic
tab <- matrix(c(p,W), ncol=2,nrow=1,byrow=T)
colnames(tab) <- c("pval", "Wstat")
tab

pval,Wstat
0.004035848,0.919403409


We reject the null hypothesis that the data are normally distributed.

c.	(1 point) Use either SAS or R to determine if the daily temperatures from the 15 randomly selected days within each of the three winter seasons follow a normal distribution. Use α = 0.01. Compare with the results from b.) and discuss which procedure is correct (i.e. either the diagnostics conducted in b.) or the diagnostics conducted in c.) ) for determining if the assumptions of normality are satisfied for an ANOVA. The R code and SAS code provided in the lecture notes and previous homework can be adapted to conduct this analysis. 


In [98]:

ddply(.data = temp_data_2,.variables = .(Winter.Season), summarise, 
      pval=shapiro.test(Maximum.Temp)$p.value)


,Winter.Season,pval
1,2013-2014,0.07404333
2,2014-2015,0.9578433
3,2015-2016,0.7984743


The assumptions of anova are that the errors $\epsilon_{ij}$ are normally distributed. Thus, we know that each observation within should be normally distributed, but each $i^{th}$ group could have a different mean, and therefore, testing for normality for several normally distributed treatments together could cause a test of normality to reject. One should perform a test of normality on either the residuals or on the individual groups.

d.	(2 points) Conduct an ANOVA and show all of your work. You can use Excel or R to help with the arithmetic, but make sure you understand how each part is calculated. Show the null and alternative hypotheses of the treatment, all degrees of freedom, all sums of squares, all mean squares, the test statistic (i.e., F calculated), the F critical value at α = 0.05 and give a complete conclusion. 


In [99]:
colnames(temp_data_2)<-c("season", "max_temp")
anova_mod <- lm(max_temp ~ season, data=temp_data_2)
anova(anova_mod)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
season,2,2576.503,1288.251,6.191893,0.004399231
Residuals,42,8738.289,208.0545,NA,NA


As anticipated, we get degrees of freedom of $2$ and $42$ for both the treatment effect and the residuals. For the null hypothesis

$$
H_0: \tau_1=\tau_2=\tau_3 = 0
$$

we use the F-test 

$$
F= \frac{MST}{MSE} = \frac{SST/df_{treat}}{SSE/df_{error}} = \frac{	2576.503/2}{8738.289/42} \approx 6.2
$$

The F-distribution has an expected value of $d_2/(d_2-2)$, that is, $42/40 \approx 1$. So, if the season had no effect on the temperature, we would expect the f test to have a value of approximately $1$. We obviously reject the anova F-test with $\alpha=0.01$ and thus conclude that we have sufficient evidence to support the assertion that the temperatures differed across seasons. 